<a href="https://colab.research.google.com/github/iskra3138/colab_seminar/blob/master/colab%EC%97%90%EC%84%9C_KorQuad_%ED%95%99%EC%8A%B5%ED%95%98%EA%B8%B02_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab에서 KorQuAD 학습하기 2
- TPU instance와 Google Drive, Google Cloud 활용
- Google Cloud 계정이 있어야만 작동하는 코드입니다.
- TPU로 학습시 약 20-`30분 소요`

In [0]:
## timezone을 한국시간으로 변경하기
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Asia/Seoul /etc/localtime
!date

## 구글 클라우드 마운트

In [0]:
# Google Cloud 계정 인증
from google.colab import auth
auth.authenticate_user()

In [0]:
# project ID 확인
!gcloud projects list

- PROJECT에는 위 셀 output을 참고해서 작성
- PRE_BUCKET에는 pre_trained model 이 저장될 storage의 이름 "gs://name"
- OUT_BUCKET에는 output을 저장할 storage의 이름 "gs://name"

In [0]:
PROJECT = "" #@param {type:"string"}
PRE_BUCKET = "gs://" #@param {type:"string"}
OUT_BUCKET = "gs://" #@param {type:"string"}

In [0]:
# pretrained files이 저장될 공간(버킷)과 output이 저장될 공간(버킷) 생성

!gsutil mb -p {PROJECT} {PRE_BUCKET}
!gsutil mb -p {PROJECT} {OUT_BUCKET}

## 구글 드라이브 마운트

In [0]:
from google.colab import drive
drive.mount('/gdrive') # 'gdrive'라는 이름으로 Google Drive를 Mount하겠다!

- Google Drive의 최상위 Directory는 '위에서 선언한 이름/My Drive'가 됨

### 실험 공간 생성

In [0]:
## 실험 공간 선언
path='/gdrive/My Drive/KorQuAD'

In [0]:
# 실험을 위한 Directory 생성
import os
if not os.path.exists(path):
  os.makedirs(path)

In [0]:
# 실험 Directory로 이동 및 확인
os.chdir(path)
os.getcwd()

### 필요한 파일 다운 받기
KorQuAD 학습하기1-GPU에서 파일들을 이미 받았다면 생략

In [0]:
# pre-train된 모델 다운받기
!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip

In [0]:
# 압축 풀고 압축파일 삭제
!unzip multi_cased_L-12_H-768_A-12.zip -d pretrained_files
!rm multi_cased_L-12_H-768_A-12.zip

In [0]:
# KorQuAD Data 다운받기

!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
!wget https://korquad.github.io/dataset/evaluate-v1.0.py

In [0]:
# 학습에 필요한 코드 다운받기
!git clone https://github.com/google-research/bert.git ./bert_files

### TPU학습을 위해 필요한 파일들을 구글드라이브에서 구글클라우드로 옮기기

In [0]:
## 코드 실행 위치 확인
import os
if not os.getcwd()==path:
  os.chdir(path)
  os.getcwd()

In [0]:
pre_drive='{}/pretrained_files'.format(path)
print (pre_drive)

In [0]:
## 로컬 -> 버킷으로 파일 복사(=업로드)
!gsutil cp -r '{pre_drive}' '{PRE_BUCKET}'

### TPU 주소 확인하고 학습 실행
- 재실행시 TPU 주소가 변경됨

In [0]:
import tensorflow as tf
import numpy as np
import os

try:
  device_name = os.environ['COLAB_TPU_ADDR']
  TPU_ADDRESS = 'grpc://' + device_name
  print('Found TPU at: {}'.format(TPU_ADDRESS))

except KeyError:
  print('TPU not found')

In [0]:
## 각 파일들의 경로 잡아주고 TPU 학습
!nohup python ./bert_files/run_squad.py \
  --vocab_file={PRE_BUCKET}/pretrained_files/multi_cased_L-12_H-768_A-12/vocab.txt \
  --bert_config_file={PRE_BUCKET}/pretrained_files/multi_cased_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint={PRE_BUCKET}/pretrained_files/multi_cased_L-12_H-768_A-12/bert_model.ckpt \
  --do_train=True \
  --train_file=./KorQuAD_v1.0_train.json \
  --do_predict=True \
  --predict_file=./KorQuAD_v1.0_dev.json \
  --train_batch_size=16 \
  --learning_rate=3e-5 \
  --num_train_epochs=2.0 \
  --max_seq_length=256 \
  --doc_stride=128 \
  --output_dir={OUT_BUCKET} \
  --use_tpu=True \
  --tpu_name=$TPU_ADDRESS \
  --do_lower_case=False \
  > log_tpu.txt &

In [0]:
### process 확인
##### python2 ./bert_files/run_squad.py 로 시작하는 process가 있으면 background에서 실행 중
!ps -ef | grep bert_files

In [0]:
### 학습을 강제로 죽이고 싶으면 위에서 찾은 process ID 입력
#!kill -9 PID

### log monitoring

In [0]:
!tail -f log_tpu.txt

In [0]:
## predictions.json 파일이 있으면 학습/평가가 완료된 것임
!!gsutil ls {OUT_BUCKET}

### Tensorboard 실행 
- 학습 중에 실행하다 보면 학습이 죽는 현상 발생. 학습이 완료되면 실행할 것을 추천

In [0]:
if not os.path.isfile('ngrok-stable-linux-amd64.zip'):
  !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
  !unzip ngrok-stable-linux-amd64.zip
elif os.path.exists('ngrok') :
  !rm ./ngrok

In [0]:
LOG_DIR = {OUT_BUCKET}
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### 성능 평가를 위해 필요한 파일들을 구글 클라우드에서 구글드라이브로 다운로드

In [0]:
## 버킷 -> 로컬로 파일 복사(=다운로드)

!mkdir temp_tpu # ouput을 저장할 폴더 생성

!gsutil cp {OUT_BUCKET}/predictions.json ./temp_tpu/predictions.json

In [0]:
# 평가
!python ./evaluate-v1.0.py ./KorQuAD_v1.0_dev.json ./temp_tpu/predictions.json

### 구글클라우드 스토리지에 저장된 파일들 삭제

In [0]:
## 생성된 bucket 삭제
!gsutil rm -r {PRE_BUCKET}
!gsutil rm -r {OUT_BUCKET}

### log 파일을 local로 다운로드

In [0]:
from google.colab import files

files.download('./log_tpu.txt')

## Clean Up
Before running the next exercise, run the following cell to terminate the kernel and free memory resources:

In [0]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)